**Import library**

In [1]:
import pandas as pd
import psycopg2
import json
import pymssql
from sqlalchemy import create_engine, text
import ast

import os
from dotenv import load_dotenv
from pathlib import Path

import sys
sys.path.append(str(Path(os.getcwd()).parent / "service"))
from database import DatabaseService

**Locate env path**

In [2]:
script_dir = Path(os.getcwd()).parent
env_path = script_dir / '.env'
load_dotenv(env_path)

print(f'Script Path:\t{script_dir}\nenv Path:\t{env_path}\nLoad env:\t{load_dotenv(env_path)}')

Script Path:	c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester
env Path:	c:\Users\Admins\Desktop\Tae\work\azure-sql-2025-tester\.env
Load env:	True


**Connection Database**

In [3]:
server = os.getenv('doi_server')
port = os.getenv('doi_port')
database = os.getenv('doi_database')
username = os.getenv('doi_username')
password = os.getenv('doi_password')

In [4]:
conn = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=server,
    port=port
)
conn

<connection object at 0x0000021D25EC28C0; dsn: 'user=opt_service password=xxx dbname=doi_operation host=10.10.10.112 port=5432', closed: 0>

In [5]:
cur = conn.cursor()
cur

<cursor object at 0x0000021D46C86BA0; closed: 0>

In [98]:
ibs_intell_query = """
SELECT DISTINCT
	i.* 
FROM 
	ibs_intell i 
JOIN
	ibs_intell_vector iv ON i.intell_id = iv.intell_id AND iv.is_active = True
WHERE 
	(iv.model_embedding = 'wordvector_BAAI' OR iv.model_embedding = 'kornwtp/ConGen-model-XLMR')
	AND i.is_publish = TRUE
	AND i.is_active = TRUE
"""

In [6]:
ibs_intell_vector_query = """
SELECT 
	iv.* 
FROM 
	ibs_intell i 
JOIN
	ibs_intell_vector iv ON i.intell_id = iv.intell_id AND iv.is_active = True
WHERE 
	(iv.model_embedding = 'wordvector_BAAI' OR iv.model_embedding = 'kornwtp/ConGen-model-XLMR')
	AND i.is_publish = TRUE
	AND i.is_active = TRUE
"""

In [46]:
cur.execute(ibs_intell_vector_query)

In [47]:
rows = cur.fetchall()
colnames = [desc[0] for desc in cur.description]
df = pd.DataFrame(rows, columns=colnames)
df

,intell_vector_id,intell_id,vector,create_by,update_by,delete_by,create_at,update_at,delete_at,is_active,nlp_id,intell_vector_type_id,model_embedding,sentence,system_name
0,55ca94ac-5a65-4c9b-a5d5-5923dd65abb1,e3e122fe-0705-47d7-8c4b-2d2d9ecf6f7c,"[0.019757796,8.7733555e-05,-0.04523224,0.00233...",None,None,None,2024-12-11 08:57:58.390253,2025-02-11 03:03:42.184137,None,True,667cdbc0a6b940c7269b7df7,8,wordvector_BAAI,None,None
1,754a5b38-988f-4691-9655-ff376c9a094e,1f242d0e-1afd-4a28-b3c1-282d992c4f8e,"[-0.02983131,-0.0064043696,-0.03684028,-0.0049...",None,None,None,2024-12-11 08:57:58.390253,2025-02-11 03:03:42.184137,None,True,667cdbc1d1f3f17b3b335d34,10,wordvector_BAAI,None,None
2,847e973b-5d79-46df-b923-2e9718817c3a,1f242d0e-1afd-4a28-b3c1-282d992c4f8e,"[-0.018671503,0.0038923493,-0.026133837,0.0085...",None,None,None,2024-12-11 08:57:58.390253,2025-02-11 03:03:42.184137,None,True,667cdbc1d1f3f17b3b335d34,9,wordvector_BAAI,None,None
3,b00be91c-89b8-4e53-9a02-f7a55d5717c6,1f242d0e-1afd-4a28-b3c1-282d992c4f8e,"[-0.02983131,-0.0064043696,-0.03684028,-0.0049...",None,None,None,2024-12-11 08:57:58.390253,2025-02-11 03:03:42.184137,None,True,667cdbc1d1f3f17b3b335d34,7,wordvector_BAAI,None,None
4,624f1be0-d315-4aae-9395-144bcb06b583,1f242d0e-1afd-4a28-b3c1-282d992c4f8e,"[-0.053423032,0.050187144,0.0013174798,-0.0127...",None,None,None,2024-12-11 08:57:58.390253,2025-02-11 03:03:42.184137,None,True,667cdbc1d1f3f17b3b335d34,8,wordvector_BAAI,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181393,462213ed-9ce2-496a-ac03-02c150a9e887,6c44e597-0c21-403c-920c-496bc3234b08,"[0.17933162,-0.22257842,0.010934065,0.07883968...",None,None,None,2024-07-02 09:43:20.973363,2024-08-26 04:45:15.007805,None,True,None,2,kornwtp/ConGen-model-XLMR,�เห็นถึงวิธีการของเกาหลีเหนือในการป้องปรามสงคร...,ibs
181394,df5361e8-d34f-4fcb-9ba2-a43d1628a1c6,6c44e597-0c21-403c-920c-496bc3234b08,"[0.007742781,-0.0143486075,0.0041142707,-0.099...",None,None,None,2024-07-02 09:43:20.973363,2024-08-26 04:45:15.007805,None,True,None,2,kornwtp/ConGen-model-XLMR,ม่ทราบประเภทอย่างน้อย ๑ ลูก จากเรือดำน้ำ\nของเ...,ibs
181395,e2e5f328-b48b-46e0-9341-be6c066d788b,4fbdee34-327a-4d32-8d6e-32bcc8d15bfe,"[-0.02983131,-0.0064043696,-0.03684028,-0.0049...",None,None,None,2024-12-11 08:57:58.390253,2025-02-11 03:03:42.184137,None,True,667cdbdea6b940c7269b7e5d,7,wordvector_BAAI,None,None
181396,bfb4eef0-072b-4f95-97d8-1a22458d2d29,4fbdee34-327a-4d32-8d6e-32bcc8d15bfe,"[-0.02983131,-0.0064043696,-0.03684028,-0.0049...",None,None,None,2024-12-11 08:57:58.390253,2025-02-11 03:03:42.184137,None,True,667cdbdea6b940c7269b7e5d,10,wordvector_BAAI,None,None


In [50]:
mssql_database = "Vector"
conn_str = f"mssql+pymssql://{os.getenv('azure_username')}:{os.getenv('azure_password')}@{os.getenv('azure_server')}:{os.getenv('azure_port')}/{mssql_database}"
table_name = "ibs_intell_vector"

**Vector Cleansing**

In [158]:
df['vector'] = df['vector'].apply(ast.literal_eval)

In [160]:
def pad_vector(vector, target_length=1024):
    if len(vector) < target_length:
        # Pad with zeros
        return vector + [0.0] * (target_length - len(vector))
    elif len(vector) > target_length:
        # Truncate to the desired length
        return vector[:target_length]
    else:
        return vector

In [161]:
df['vector'] = df['vector'].apply(pad_vector)

In [163]:
def serialize_vector(v):
    return json.dumps([round(float(x), 6) for x in v])  # Optional rounding


In [ ]:
df['vector'] = df['vector'].apply(serialize_vector)

**End of Vector Cleansing**

In [166]:
try:
    # Create the SQLAlchemy engine
    engine = create_engine(conn_str)

    # Establish a connection explicitly using engine.connect()
    with engine.connect() as conn:
        # Start a transaction using conn.begin()
        with conn.begin():

            # Insert the DataFrame into the SQL Server table
            df.to_sql(table_name, conn, schema='dbo', if_exists='append', index=False)
            
    print("Data inserted successfully into the SQL Server table.")
    # conn = conn.close()
except Exception as e:
    print(f"An error occurred: {e}")

Data inserted successfully into the SQL Server table.
